# Homework 2

In [ ]:
##### IMPORTS
import numpy as np
import math

In [ ]:
### EQUATION 1 SHOULD HAVE A "+" BEFORE THE SUM

def RegressionAtHome(X_train, Y_train, x_pred, tau):


# Gradient is first partial derivatives of the cost function
# Hessian Matrix is second partial derivatives of the cost function
# Newton's method uses

def magnitude(vector):
    return math.sqrt(sum(pow(element, 2) for element in vector))

def calc_weight(x_q, x_i, tau):
    return

## Problem 2

1. Soft-Margin Linear SVM, C=.02 corresponds to #4. This is a linear model with larger margins, so the C value is lower.
2. Soft-Margin Linear SVM, C=20 corresponds to #3. This is a linear model with smaller margins, so the C value is higher.
3.